# Bus Intersection Major Transit Stops, Spatial Pathway

In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(200_000_000_000)

import intake
import geopandas as gpd
import pandas as pd
import numpy as np
from calitp_data_analysis.gcs_geopandas import GCSGeoPandas
from calitp_data_analysis import geography_utils
import calitp_data_analysis.magics

from shared_utils import webmap_utils, gtfs_utils_v2, rt_dates
import branca

In [2]:
from update_vars import analysis_date, GCS_FILE_PATH, INTERSECTION_BUFFER_METERS, MS_TRANSIT_THRESHOLD, SHARED_STOP_THRESHOLD

In [3]:
gcsgp = GCSGeoPandas()

In [5]:
catalog = intake.open_catalog('catalog.yml')

## Analysis Segments and Key Stops

* We use 1,250 meter analysis segments cut from GTFS shapes.
* In each segment, we identify the stop with the highest frequency and use it to assign frequency to the segment.

In [6]:
hqta_segments = catalog.hqta_segments.read()

In [7]:
path = f'{GCS_FILE_PATH}all_bus.parquet'

In [9]:
max_arrivals_by_stop = pd.read_parquet(f"{GCS_FILE_PATH}max_arrivals_by_stop.parquet")

In [10]:
gdf = gcsgp.read_parquet(path)

stops = gcsgp.read_parquet(f"{GCS_FILE_PATH}stops_with_lookback.parquet")

stops = stops[['stop_id', 'stop_name', 'analysis_date',
      'schedule_gtfs_dataset_key', 'analysis_name', 'geometry']]

stops = stops.rename(columns={'geometry': 'stop_geometry'})

gdf = gdf.merge(stops, on = ['stop_id', 'schedule_gtfs_dataset_key'])
gdf = gdf[~gdf['circuitous_segment']]

In [11]:
map1 = gdf.copy()[['route_id', 'stop_id', 'geometry',
   'fwd_azimuth_360', 'circuitous_segment', 'hq_transit_corr',
   'ms_precursor', 'analysis_name']]

In [12]:
# Source - https://stackoverflow.com/a
# Posted by mkrieger1, modified by community. See post 'Timeline' for change history
# Retrieved 2025-12-08, License - CC BY-SA 4.0

azimuth_cmap = branca.colormap.LinearColormap(
        colors=list(branca.colormap.linear.viridis.colors) + list(reversed(branca.colormap.linear.viridis.colors)),
        vmin=0, vmax=360
)  # this will correctly show 0 and 360 as close together
azimuth_cmap.caption = '360-degree azimuth (heading)'

In [13]:
%%capture

webmap_utils.export_legend(azimuth_cmap, 'azimuth_viridis_360a.svg', inner_labels=list(range(72, 360, 72)))

legend written to calitp-map-tiles/azimuth_viridis_360a.svg, public_url https://storage.googleapis.com/calitp-map-tiles/azimuth_viridis_360a.svg


In [14]:
segment_state = webmap_utils.set_state_export(map1, filename = 'hqta_segments', cache_seconds=0,
                                     map_title='HQTA Segments', overwrite=True, color_col='fwd_azimuth_360',
                                     cmap = azimuth_cmap, legend_url='https://storage.googleapis.com/calitp-map-tiles/azimuth_viridis_360a.svg')

/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:128: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


In [15]:
map2 = gdf.copy()[['stop_id', 'stop_name', 'am_max_trips_hr',
                  'pm_max_trips_hr', 'route_dir_count', 'analysis_name',
                  'stop_geometry']].set_geometry('stop_geometry')

In [16]:
map2['color'] = [(0,0,0)] * len(map2)

In [17]:
segment_stop_state = webmap_utils.set_state_export(map2, filename = 'hqta_segment_key_stops', cache_seconds=0,
                             existing_state=segment_state, map_title='Key Stops and Segments', overwrite=True,
                                                  manual_centroid=[37.336813156889704, -121.88911054161129])

In [18]:
webmap_utils.render_spa_link(segment_stop_state['spa_link'])

<a href="https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJIUVRBIFNlZ21lbnRzIiwgInVybCI6ICJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY2FsaXRwLW1hcC10aWxlcy90ZXN0aW5nL2hxdGFfc2VnbWVudHMuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41fX0sIHsibmFtZSI6ICJLZXkgU3RvcHMgYW5kIFNlZ21lbnRzIiwgInVybCI6ICJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY2FsaXRwLW1hcC10aWxlcy90ZXN0aW5nL2hxdGFfc2VnbWVudF9rZXlfc3RvcHMuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41fX1dLCAibGF0X2xvbiI6IFszNy4zMzY4MTMxNTY4ODk3MDQsIC0xMjEuODg5MTEwNTQxNjExMjldLCAiem9vbSI6IDEzLCAibGVnZW5kX3VybCI6ICJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY2FsaXRwLW1hcC10aWxlcy9hemltdXRoX3ZpcmlkaXNfMzYwYS5zdmcifQ==" target="_blank">Open Full Map in New Tab</a>

In [19]:
webmap_utils.display_spa_map(segment_stop_state['spa_link'])

In [20]:
pairs = pd.read_parquet(f"{GCS_FILE_PATH}pairwise.parquet")

## Spatial Intersections

* We use an azimuth (heading) approach to find intersections, segments are considered intersecting if they diverge at a 45-degree angle or greater. Our goal is to identify locations where riders have access to multiple frequent routes that can take them in different directions.
* Intersections are colored in red in the map below.

In [21]:
intersect = gcsgp.read_parquet(f"{GCS_FILE_PATH}all_intersections.parquet")

In [22]:
by_segment = intersect.dissolve(['hqta_segment_id']).reset_index(drop=False)

In [23]:
by_segment['color'] = [(140, 0, 0)] * len(by_segment)

In [24]:
segment_intersect_state = webmap_utils.set_state_export(by_segment, filename = 'hqta_intersection_areas', cache_seconds=0,
                             existing_state=segment_state, map_title='Segments with Intersections', overwrite=True,
                                                       manual_centroid=[37.336813156889704, -121.88911054161129])

In [25]:
webmap_utils.render_spa_link(segment_intersect_state['spa_link'])

<a href="https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJIUVRBIFNlZ21lbnRzIiwgInVybCI6ICJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY2FsaXRwLW1hcC10aWxlcy90ZXN0aW5nL2hxdGFfc2VnbWVudHMuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41fX0sIHsibmFtZSI6ICJTZWdtZW50cyB3aXRoIEludGVyc2VjdGlvbnMiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL3Rlc3RpbmcvaHF0YV9pbnRlcnNlY3Rpb25fYXJlYXMuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41fX1dLCAibGF0X2xvbiI6IFszNy4zMzY4MTMxNTY4ODk3MDQsIC0xMjEuODg5MTEwNTQxNjExMjldLCAiem9vbSI6IDEzLCAibGVnZW5kX3VybCI6ICJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY2FsaXRwLW1hcC10aWxlcy9hemltdXRoX3ZpcmlkaXNfMzYwYS5zdmcifQ==" target="_blank">Open Full Map in New Tab</a>

In [26]:
webmap_utils.display_spa_map(segment_intersect_state['spa_link'])

## Intersection Buffers and Stop Groups

* We use a 500ft buffer around the spatial intersection to find physical stops associated with the intersection.
* We consider all of these physical stops to be Major Transit Stops.

In [27]:
by_segment.geometry = by_segment.buffer(INTERSECTION_BUFFER_METERS)

In [28]:
major_bus_spatial = gcsgp.read_parquet(f"{GCS_FILE_PATH}major_stop_bus.parquet")

In [29]:
intersect_buffered_state = webmap_utils.set_state_export(by_segment, filename = 'intersect_buffered', cache_seconds=0,
                           map_title='Intersecton Buffers', overwrite=True,
                                                         manual_centroid=[37.336813156889704, -121.88911054161129])

In [30]:
major_bus_spatial['color'] = [(200, 200, 255)] * len(major_bus_spatial)

In [31]:
intersect_major_state = webmap_utils.set_state_export(major_bus_spatial, filename = 'major_bus_spatial', cache_seconds=0,
                           existing_state=intersect_buffered_state, map_title='Buffered Intersections and Stop Groups', overwrite=True,
                                                         manual_centroid=[37.336813156889704, -121.88911054161129])

In [32]:
webmap_utils.render_spa_link(intersect_major_state['spa_link'])

<a href="https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJJbnRlcnNlY3RvbiBCdWZmZXJzIiwgInVybCI6ICJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY2FsaXRwLW1hcC10aWxlcy90ZXN0aW5nL2ludGVyc2VjdF9idWZmZXJlZC5nZW9qc29uLmd6IiwgInByb3BlcnRpZXMiOiB7InN0cm9rZWQiOiBmYWxzZSwgImhpZ2hsaWdodF9zYXR1cmF0aW9uX211bHRpcGxpZXIiOiAwLjV9fSwgeyJuYW1lIjogIkJ1ZmZlcmVkIEludGVyc2VjdGlvbnMgYW5kIFN0b3AgR3JvdXBzIiwgInVybCI6ICJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY2FsaXRwLW1hcC10aWxlcy90ZXN0aW5nL21ham9yX2J1c19zcGF0aWFsLmdlb2pzb24uZ3oiLCAicHJvcGVydGllcyI6IHsic3Ryb2tlZCI6IGZhbHNlLCAiaGlnaGxpZ2h0X3NhdHVyYXRpb25fbXVsdGlwbGllciI6IDAuNX19XSwgImxhdF9sb24iOiBbMzcuMzM2ODEzMTU2ODg5NzA0LCAtMTIxLjg4OTExMDU0MTYxMTI5XSwgInpvb20iOiAxM30=" target="_blank">Open Full Map in New Tab</a>

In [33]:
webmap_utils.display_spa_map(intersect_major_state['spa_link'])